In [58]:
import nltk,re
import pandas as pd
from nltk import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from pprint import pprint
from collections import Counter
from math import *
from datetime import datetime
# nltk.download('punkt')

In [59]:
def get_content(data_df):
    content=data_df.content
    content.index = pd.DatetimeIndex(content.index)
    content=content.dropna(how="any")
    return(content)
def stem_and_other_stuff(each_news):
    ps=PorterStemmer()
    return([ps.stem(word.lower()) for word in each_news if word.isalpha()])
def check_alpha_tolower(each_news):
    return([word.lower() for word in each_news if word.isalpha()])

def to_counter(this_year_content,stem=True):
    token_content=pd.Series()
    token_content=this_year_content.apply(word_tokenize)
    ps=PorterStemmer()
    if stem:        
        stemmed_content=token_content.apply(stem_and_other_stuff)
    else:
        stemmed_content=token_content.apply(check_alpha_tolower)
    content_counter = Counter()
    for news in stemmed_content:
        content_counter.update(news)
    return(stemmed_content,content_counter)


class MyStopWord:
    def __init__(self,content_counter,most_common=100,stop_word=None):
        from nltk.corpus import stopwords
        self.counter_stop_word=[word for word,time in content_counter.most_common(most_common)]
        self.user_keep=[]
        self.user_define=[]
        if stop_word:
            self.stop_word=stop_word
        else:
            self.stop_word=set(self.counter_stop_word+stopwords.words('english')) 
    def keep(self,word):
        self.user_keep.append(word)
        self.stop_word.discard(word)
    def define(self,word):
        self.user_define.append(word)
        self.stop_word.add(word)
        

In [3]:
path = "../../data/ebscohost_history_data/History_news_data_with_datetime.csv"
ebsco_df=pd.read_csv(path)

In [60]:
raw_df = pd.read_csv("../../data/crawler_news_data/oilprice_news.csv")

In [61]:
# path = "../../data/crawler_news_data/oilprice_news.csv"
# raw_df=pd.read_csv(path)
data_df=raw_df.sort_values(by="publish_datetime",ascending=True).set_index('publish_datetime')
content = get_content(data_df)

# stemming and get target news date

In [63]:
this_year_content=content["2017-11":"2018-09"]
this_year_token, this_year_counter = to_counter(this_year_content,stem=False)

effective_news_df=pd.read_csv("../../data/crude_oil_price/effective_news_date.csv")
effective_news_date = effective_news_df['date']
effective_news_date=pd.DatetimeIndex(effective_news_date)

effective_news_date

DatetimeIndex(['2017-11-02', '2018-02-05', '2018-02-06', '2018-02-07',
               '2018-02-26', '2018-03-06', '2018-03-19', '2018-04-06',
               '2018-04-09', '2018-04-16', '2018-05-03', '2018-05-08',
               '2018-05-23', '2018-05-24', '2018-05-30', '2018-05-31',
               '2018-06-20', '2018-06-21', '2018-06-22', '2018-06-25',
               '2018-06-26', '2018-07-09', '2018-07-10', '2018-07-12',
               '2018-07-13', '2018-07-19', '2018-07-20', '2018-07-30',
               '2018-08-06', '2018-08-07', '2018-08-13', '2018-09-10',
               '2018-09-27', '2018-09-28', '2018-10-03', '2018-10-09',
               '2018-10-10', '2018-10-15', '2018-10-16', '2018-10-19',
               '2018-10-22'],
              dtype='datetime64[ns]', name='date', freq=None)

# get token and target token

In [64]:
target_content = content.loc[effective_news_date.values].dropna(how="any")
other_content=content.loc[~this_year_content.index.isin(effective_news_date.values)].dropna(how="any")
# target_stem_token, target_stem_counter = to_counter(target_content)
# other_stem_token, other_stem_counter = to_counter(other_content)
target_token,target_counter=to_counter(target_content,False)
other_token,other_counter=to_counter(other_content,False)
content_token,content_counter=to_counter(content,False)

list(enumerate(content_counter.most_common(87)))

C:\Users\eiahb\AppData\Local\conda\conda\envs\env_futures\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


[(0, ('the', 115518)),
 (1, ('to', 57086)),
 (2, ('of', 54267)),
 (3, ('in', 46840)),
 (4, ('and', 42963)),
 (5, ('oil', 36877)),
 (6, ('a', 35088)),
 (7, ('is', 26485)),
 (8, ('that', 25100)),
 (9, ('for', 20768)),
 (10, ('s', 15212)),
 (11, ('from', 13995)),
 (12, ('on', 13617)),
 (13, ('as', 13179)),
 (14, ('it', 12995)),
 (15, ('are', 12222)),
 (16, ('production', 12200)),
 (17, ('be', 11237)),
 (18, ('with', 11173)),
 (19, ('by', 11145)),
 (20, ('this', 10221)),
 (21, ('has', 10201)),
 (22, ('will', 10107)),
 (23, ('at', 9579)),
 (24, ('have', 8839)),
 (25, ('its', 8084)),
 (26, ('but', 6980)),
 (27, ('not', 6968)),
 (28, ('more', 6875)),
 (29, ('an', 6518)),
 (30, ('prices', 6510)),
 (31, ('energy', 6237)),
 (32, ('was', 6014)),
 (33, ('crude', 5992)),
 (34, ('million', 5685)),
 (35, ('which', 5669)),
 (36, ('would', 4951)),
 (37, ('gas', 4930)),
 (38, ('barrels', 4880)),
 (39, ('than', 4802)),
 (40, ('been', 4582)),
 (41, ('year', 4581)),
 (42, ('opec', 4538)),
 (43, ('we', 4388

# define stop words

In [65]:
mystopword=MyStopWord(content_counter=content_counter,most_common=87)
mystopword.define('c')
mystopword.keep('demand')

# get corpus set

In [66]:
TF_OTHER_THRESHOLD=20
TF_TARGET_THRESHOLD=5

other_corpus_set=set(key for key,times in other_counter.items() if times>TF_OTHER_THRESHOLD)-mystopword.stop_word
target_corpus_set=set(key for key,times in target_counter.items() if times>TF_TARGET_THRESHOLD)-mystopword.stop_word
target_corpus_set-other_corpus_set

{'accounting',
 'aggressive',
 'alexander',
 'allies',
 'amid',
 'argues',
 'arguing',
 'aurora',
 'authority',
 'averaged',
 'baker',
 'barclays',
 'basins',
 'benchmarks',
 'berman',
 'bijan',
 'biofuel',
 'bloomberg',
 'boosted',
 'bottlenecks',
 'breakeven',
 'bullish',
 'buyer',
 'campos',
 'cartel',
 'castberg',
 'cautious',
 'cited',
 'citing',
 'click',
 'cnbc',
 'compiled',
 'compliance',
 'conocophillips',
 'consensus',
 'considerably',
 'continental',
 'correlation',
 'cunningham',
 'cuts',
 'cutting',
 'dated',
 'decide',
 'dip',
 'discount',
 'dispute',
 'distillate',
 'distillates',
 'donald',
 'downside',
 'downturn',
 'draw',
 'drillers',
 'drivers',
 'dudley',
 'e',
 'earnings',
 'ease',
 'encana',
 'ending',
 'enlarge',
 'eog',
 'epa',
 'ev',
 'evs',
 'exceeded',
 'executives',
 'exported',
 'extending',
 'faces',
 'fears',
 'flood',
 'flows',
 'fpsos',
 'friday',
 'fundamentals',
 'geiger',
 'generated',
 'glut',
 'goldman',
 'grade',
 'grades',
 'granted',
 'grows',

# pairwise experiment

In [67]:
word_pair_counts = Counter()
tokens = target_content

for tokens in target_token:
    for i in range(len(tokens) - 1):
        (w1, w2) = (tokens[i], tokens[i + 1])
        word_pair_counts[(w1, w2)] += 1
    
for pair, c in word_pair_counts.most_common(20):
    print("%s\t%s\t%d" % (pair[0], pair[1], c))

in	the	453
of	the	373
to	the	233
saudi	arabia	186
for	the	176
oil	prices	158
on	the	150
that	the	145
the	oil	144
crude	oil	143
million	bpd	135
oil	production	127
from	the	124
and	the	121
top	reads	118
reads	from	118
at	the	107
according	to	102
oil	and	99
million	barrels	93


In [68]:
def count_word_pair(target_token):
    stop_word = mystopword.stop_word
    target_word_pair = Counter()
    for tokens in target_token:
        for i in range(len(tokens) - 1):
            (w1, w2) = (tokens[i], tokens[i + 1])
            if w1 not in stop_word and w2 not in stop_word:
                target_word_pair[(w1, w2)] += 1
    return(target_word_pair)

def count_word_pair_with_windows(target_token,window_size):
    stop_word = mystopword.stop_word
    target_pair_counts = Counter()
    target_pair_distance_counts = Counter()
    for tokens in target_token:
        for i in range(len(tokens) - 1):
            for distance in range(1, window_size):
                if i + distance < len(tokens):
                    w1 = tokens[i]
                    w2 = tokens[i + distance]
                    if w1 not in stop_word and w2 not in stop_word:
                        target_pair_distance_counts[(w1, w2, distance)] += 1
                        target_pair_counts[(w1, w2)] += 1
    return(target_pair_distance_counts,target_pair_counts)

In [69]:
target_word_pair = count_word_pair(target_token)
other_word_pair = count_word_pair(other_token)
pairwise=set(target_word_pair.most_common(100)) - set(other_word_pair.most_common(100))

In [70]:
window_size = 5
target_pair_distance_counts,target_pair_counts = count_word_pair_with_windows(target_token,window_size)
other_pair_distance_counts,other_pair_counts = count_word_pair_with_windows(other_token,window_size)
pairwise_with_windows = set(target_pair_counts.most_common(100)) - set(other_pair_counts)
pairwise_with_windows_list=[pair for pair,count in pairwise_with_windows]
pairwise_with_windows_list[:]

[('tsvetana', 'reads'),
 ('oilfield', 'service'),
 ('record', 'high'),
 ('coming', 'months'),
 ('kinder', 'morgan'),
 ('third', 'quarter'),
 ('north', 'sea'),
 ('west', 'texas'),
 ('told', 'bloomberg'),
 ('donald', 'trump'),
 ('investment', 'bank'),
 ('lowest', 'level'),
 ('wall', 'street'),
 ('according', 'bloomberg'),
 ('brent', 'wti'),
 ('supply', 'demand'),
 ('enlarge', 'enlarge'),
 ('tsvetana', 'paraskova'),
 ('gasoline', 'inventories'),
 ('week', 'october'),
 ('russia', 'arabia'),
 ('international', 'agency'),
 ('sources', 'reuters'),
 ('alexander', 'novak'),
 ('baker', 'hughes'),
 ('nick', 'cunningham'),
 ('eia', 'report'),
 ('sources', 'told'),
 ('level', 'since'),
 ('arabia', 'russia'),
 ('neutral', 'zone'),
 ('net', 'position'),
 ('sanctions', 'iran'),
 ('middle', 'eastern'),
 ('cunningham', 'reads'),
 ('information', 'administration'),
 ('independent', 'refiners'),
 ('service', 'providers'),
 ('gulf', 'mexico'),
 ('first', 'quarter'),
 ('net', 'long'),
 ('paraskova', 'reads'

In [71]:
len(pairwise_with_windows_list)

100

In [72]:
pairwise_df = pd.DataFrame(pairwise_with_windows_list)
pairwise_df.to_csv("../../data/wordpair_result/bigram_result_this_year_oilprice.csv",index=False)

## Word to Vector

In [73]:
# pairwise_chose_df = pd.read_excel("../../data/wordpair_result/bigram_result_excel.xlsx")
# pairwise_chosen_tuple = [tuple(x[:2]) for x in pairwise_chose_df.values]

In [74]:
def word_to_vector(this_year_token,pairwise_with_windows_list,mystopword):
    this_year_vs=[]
    # pairwise_with_windows_list = pairwise_chosen_tuple
    for tokenized_article in this_year_token:
        finder = nltk.BigramCollocationFinder.from_words([word for word in tokenized_article if word not in mystopword.stop_word],window_size=window_size)
        this_vs= {key: 0 for key in pairwise_with_windows_list}
        for pair,times in finder.ngram_fd.items():
            if pair in this_vs.keys():
                this_vs[pair]=times
        this_year_vs.append(this_vs)            
    return(this_year_vs)

In [75]:
this_year_vs = word_to_vector(this_year_token,pairwise_with_windows_list,mystopword)
this_year_vs_df=pd.DataFrame(this_year_vs)
this_year_vs_df=this_year_vs_df.set_index(pd.DatetimeIndex(this_year_content.index))

print("target:",this_year_vs_df.loc[effective_news_date.values].sum(axis=1).mean())
print("other:",this_year_vs_df.loc[~this_year_content.index.isin(effective_news_date.values)].sum(axis=1).mean())

target: 11.96039603960396
other: 8.836978131212724


C:\Users\eiahb\AppData\Local\conda\conda\envs\env_futures\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """


In [76]:
oilprice_pairwise_df = pd.DataFrame({"pairwise": pairwise_with_windows_list})
oilprice_pairwise_df['pairwise1']=[x for x,y in oilprice_pairwise_df.pairwise]
oilprice_pairwise_df['pairwise2']=[y for x,y in oilprice_pairwise_df.pairwise]
oilprice_pairwise_df.to_csv("../../data/wordpair_result/oilprice_pairwise_df.csv",sep=";",index = False)

In [21]:
def content_to_pairwise_vector(test_content,window_size,pairwise_with_windows_list):
    test_token,test_counter = to_counter(test_content,False)
    test_pairwise_with_distance,test_pairwise = count_word_pair_with_windows(test_token,window_size)
    test_vs = word_to_vector(test_token,pairwise_with_windows_list,mystopword)
    test_vs_df = pd.DataFrame(test_vs,index = test_content.index)
    return(test_vs_df)

In [22]:
# 手動標的
# test_df = raw_df.loc[:75]
# test_content = test_df.content
# test_vs_df = content_to_pairwise_vector(test_content,window_size,pairwise_with_windows_list)

In [77]:
this_year_vs_df['tags'] = 0
this_year_vs_df.loc[this_year_vs_df.index.isin(effective_news_date.values),'tags']=1

In [78]:
this_year_vs_df.to_csv("../../data/train_test_dataset/this_year_oilprice_train.csv",index = False)
test_vs_df.to_csv("../../data/train_test_dataset/this_month_oilprice_test.csv",index = False)

In [79]:
this_year_vs_df

,"(according, bloomberg)","(according, data)","(alexander, novak)","(arabia, capacity)","(arabia, russia)","(average, daily)","(baker, hughes)","(brent, barrel)","(brent, wti)","(cash, flow)",...,"(trump, administration)","(tsvetana, paraskova)","(tsvetana, reads)","(united, states)","(venezuelan, exports)","(wall, street)","(week, october)","(west, texas)","(wood, mackenzie)",tags
publish_datetime,,,,,,,,,,,,,,,,,,,,,
2017-11-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2017-11-01,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,1,0
2017-11-02,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,1,0,1
2017-11-02,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,1
2017-11-02,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
2017-11-03,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2017-11-03,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
2017-11-03,0,0,0,0,0,0,2,0,0,0,...,0,0,0,3,0,0,1,0,0,0
2017-11-03,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Use Effective date as test data

In [25]:
effective_date = pd.read_csv("effective_news_date_from_2013.csv")
test_target_date = effective_date.loc[effective_date.date>"2018-10-01","date"]

In [28]:
test_content = content["2018-10-01":]
test_vs_df = content_to_pairwise_vector(test_content,window_size,pairwise_with_windows_list)
test_vs_df['tags'] = 0

test_vs_df.loc[test_vs_df.index.isin(test_target_date)] =1
test_vs_df.to_csv("../../data/train_test_dataset/this_month_oilprice_tag_by_date.csv",index=False)

In [29]:
test_vs_df

,"(according, data)","(arabia, capacity)","(arabia, russia)","(average, daily)","(brent, barrel)","(brent, wti)","(cash, flow)","(china, imports)","(click, click)","(click, enlarge)",...,"(told, reuters)","(trade, war)","(trans, mountain)","(trump, administration)","(tsvetana, paraskova)","(tsvetana, reads)","(united, states)","(week, october)","(wood, mackenzie)",tags
publish_datetime,,,,,,,,,,,,,,,,,,,,,
2018-10-01,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-10-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-10-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,3,0
2018-10-01,0,0,0,0,2,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2018-10-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
2018-10-02,0,0,0,0,0,0,0,2,0,0,...,0,2,0,0,0,0,0,0,0,0
2018-10-02,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018-10-02,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
2018-10-02,0,1,0,0,2,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
